<a href="https://colab.research.google.com/github/VimalChavda/DataScience/blob/master/Copy_of_OptimizeKerasTunerForCNN_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow


In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

2.3.0


In [ ]:
from tensorflow import keras


In [ ]:
fasion_dataset=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fasion_dataset.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
    model = keras.Sequential([
    keras.layers.Conv2D(
    filters=hp.Int('conv_1_filter',min_value=32,max_value=128, step=16),
    kernel_size=hp.Choice('conv_1_kernal',values=[3,5]),
    activation='relu',
    input_shape=(28,28,1)
    ),

    keras.layers.Conv2D(
    filters=hp.Int('conv_2_filter',min_value=32,max_value=128, step=16),
    kernel_size=hp.Choice('conv_2_kernal',values=[3,5]),
    activation='relu',
    input_shape=(28,28,1)),
    keras.layers.Flatten(),
    keras.layers.Dense(units=hp.Int('dense_1_layer',min_value=32,max_value=128,step=16),
                       activation='relu'),
    keras.layers.Dense(10,activation='softmax')])
    model.compile(optimizer=keras.optimizers.Adam( hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 8.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=edf40342d05d53d631a092302df9186fbf76f09467ae65a30377015b8c7e3c5a
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15357 sha256=b8f23d148c83baa855b157aca1d0e420ffc4ea6d99fefb13748475e7d5943edd
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search= RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="MNIST fasion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 25s]
val_accuracy: 0.9078333377838135

Best val_accuracy So Far: 0.9108333587646484
Total elapsed time: 00h 02m 23s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 80)        46160     
_________________________________________________________________
flatten (Flatten)            (None, 46080)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                3686480   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 3,734,090
Trainable params: 3,734,090
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1160 - accuracy: 0.9574 - val_loss: 0.2591 - val_accuracy: 0.9147
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0784 - accuracy: 0.9710 - val_loss: 0.3133 - val_accuracy: 0.9107
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0526 - accuracy: 0.9809 - val_loss: 0.3475 - val_accuracy: 0.9143
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0388 - accuracy: 0.9862 - val_loss: 0.4503 - val_accuracy: 0.9040
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0272 - accuracy: 0.9903 - val_loss: 0.4688 - val_accuracy: 0.9107
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0249 - accuracy: 0.9912 - val_loss: 0.5136 - val_accuracy: 0.9103
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0201 - accuracy: 0.9929 - val_loss: 0.5535 - val_accuracy

In [ ]:
model.save("kerastunerCNNmodel.h5")